# Customer Complaints Analysis

## Setup

In [10]:
%reload_ext autoreload
%autoreload 2

import os
jp = os.path.join
import sys
import json
import pandas as pd
import glob
import datetime
T_now = datetime.datetime.now
import logging

#logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logging.basicConfig(level=logging.DEBUG)



from reviewanalysis import (
    GPT,
    Embedding,
    print_shape,
    load_customer_reviews,
    filter_by_styles,
    create_product_dict,
)

from reviewanalysis.gpt_class import DEFAULT_PROMPT, CATEGORY_PROMPT

# Prepare for ChatGPT
gptchat = GPT()

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/opt/anaconda3/envs/conda-python3.12/lib/python3.12/site-packages/certifi/cacert.pem'


In [11]:
gptchat.list_models()

DEBUG:openai._base_client:Request options: {'method': 'get', 'url': '/models', 'post_parser': <function SyncAPIClient._request_api_list.<locals>._parser at 0x7fc9ce7af6a0>, 'json_data': None}
DEBUG:openai._base_client:Sending HTTP Request: GET https://api.openai.com/v1/models
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=5.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7fc9ceb3a8d0>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x7fc9cf5f2f50> server_hostname='api.openai.com' timeout=5.0
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7fc9cf3f88f0>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'GET']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'GET']>
DEBUG

['gpt-4-turbo-2024-04-09',
 'tts-1-1106',
 'gpt-4o-mini-2024-07-18',
 'dall-e-2',
 'gpt-4o-mini',
 'whisper-1',
 'gpt-3.5-turbo-instruct',
 'tts-1-hd-1106',
 'gpt-3.5-turbo',
 'gpt-3.5-turbo-0125',
 'babbage-002',
 'davinci-002',
 'dall-e-3',
 'gpt-4o-realtime-preview',
 'gpt-4o-realtime-preview-2024-10-01',
 'gpt-4o-2024-08-06',
 'gpt-4o',
 'tts-1-hd',
 'tts-1',
 'chatgpt-4o-latest',
 'gpt-4-turbo-preview',
 'gpt-4-1106-preview',
 'text-embedding-ada-002',
 'gpt-3.5-turbo-16k',
 'gpt-4-turbo',
 'text-embedding-3-small',
 'text-embedding-3-large',
 'gpt-3.5-turbo-1106',
 'gpt-4o-audio-preview',
 'gpt-4o-2024-05-13',
 'gpt-4o-audio-preview-2024-10-01',
 'gpt-4-0613',
 'gpt-4-0125-preview',
 'gpt-4',
 'gpt-3.5-turbo-instruct-0914']

In [15]:
### Query OpenAI
prompt = """
What is the capitol of France?
"""
response = gptchat.client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [
        {"role": "user", "content": prompt}
    ],
    temperature = 0,
)
display(response)
            
content = response.choices[0].message.content

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': '\nWhat is the capitol of France?\n'}], 'model': 'gpt-3.5-turbo', 'temperature': 0}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=5.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7fc9ceb3ac30>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x7fc9cf5f2f50> server_hostname='api.openai.com' timeout=5.0
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7fc9ceb38350>
DEBUG:httpcore.http11:send_request_headers.started request=<Reque

ChatCompletion(id='chatcmpl-AQdsKfUhXv04MrRw0eUsAIXouA3pS', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The capital of France is Paris.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1730913172, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=7, prompt_tokens=16, total_tokens=23, completion_tokens_details=CompletionTokensDetails(audio_tokens=0, reasoning_tokens=0, accepted_prediction_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [16]:
print(content)

The capital of France is Paris.


# Data

In [19]:
DATA_FILE = '/data/public/ConsumerComplaintDatabase/rows.csv'
df = pd.read_csv(DATA_FILE, nrows=20)
print_shape(df)

INFO:root:Number of records: 20, number of fields: 18


In [18]:
df.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,05/10/2019,Checking or savings account,Checking account,Managing an account,Problem using a debit or ATM card,NaN,NaN,NAVY FEDERAL CREDIT UNION,FL,328XX,Older American,NaN,Web,05/10/2019,In progress,Yes,NaN,3238275
1,05/10/2019,Checking or savings account,Other banking product or service,Managing an account,Deposits and withdrawals,NaN,NaN,BOEING EMPLOYEES CREDIT UNION,WA,98204,NaN,NaN,Referral,05/10/2019,Closed with explanation,Yes,NaN,3238228
2,05/10/2019,Debt collection,Payday loan debt,Communication tactics,Frequent or repeated calls,NaN,NaN,CURO Intermediate Holdings,TX,751XX,NaN,NaN,Web,05/10/2019,Closed with explanation,Yes,NaN,3237964
3,05/10/2019,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Old information reappears or never goes away,NaN,NaN,Ad Astra Recovery Services Inc,LA,708XX,NaN,NaN,Web,05/10/2019,Closed with explanation,Yes,NaN,3238479
4,05/10/2019,Checking or savings account,Checking account,Managing an account,Banking errors,NaN,NaN,ALLY FINANCIAL INC.,AZ,85205,NaN,NaN,Postal mail,05/10/2019,In progress,Yes,NaN,3238460


In [22]:
prompt = """
Analyze the provided complaint delimited by <> and extract service-related
aspects, sentiments (positive/negative/neutral), and justifications.
Format the result as JSON object with the following structure: a key named 'aspect'
containing a nested object with keys for each aspect, and each aspect having an object with
'sentiment' and 'justification' keys.
Exclude non-product discussions. If no sentiment is expressed for an aspect, return
'neutral' as the sentiment and 'No information provided' as the value. If information
can't be retrieved, return an empty JSON object.
Review: <{review}>
"""

for i, row in df.iterrows():
    review = f"{row['Issue']} {row['Sub-issue']}"
    prmt = prompt.format(review = review)
    print(prmt)
    response = gptchat.client.chat.completions.create(
        model = "gpt-3.5-turbo",
        messages = [
            {"role": "user", "content": prompt}
        ],
        temperature = 0,
    )
    content = response.choices[0].message.content
    print(f">>>\n{content}\n\n")
    
    

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "\nAnalyze the provided complaint delimited by <> and extract service-related\naspects, sentiments (positive/negative/neutral), and justifications.\nFormat the result as JSON object with the following structure: a key named 'aspect'\ncontaining a nested object with keys for each aspect, and each aspect having an object with\n'sentiment' and 'justification' keys.\nExclude non-product discussions. If no sentiment is expressed for an aspect, return\n'neutral' as the sentiment and 'No information provided' as the value. If information\ncan't be retrieved, return an empty JSON object.\nReview: <{review}>\n"}], 'model': 'gpt-3.5-turbo', 'temperature': 0}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG


Analyze the provided complaint delimited by <> and extract service-related
aspects, sentiments (positive/negative/neutral), and justifications.
Format the result as JSON object with the following structure: a key named 'aspect'
containing a nested object with keys for each aspect, and each aspect having an object with
'sentiment' and 'justification' keys.
Exclude non-product discussions. If no sentiment is expressed for an aspect, return
'neutral' as the sentiment and 'No information provided' as the value. If information
can't be retrieved, return an empty JSON object.
Review: <Managing an account Problem using a debit or ATM card>



DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Wed, 06 Nov 2024 17:27:52 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'user-nw5aseigyjbyzll3ouyazpnn'), (b'openai-processing-ms', b'1774'), (b'openai-version', b'2020-10-01'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'200000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'199832'), (b'x-ratelimit-reset-requests', b'8.64s'), (b'x-ratelimit-reset-tokens', b'50ms'), (b'x-request-id', b'req_b20904a06024ffa53e2883650f3da024'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'CF-Cache-Status', b'DYNAMIC'), (b'Set-Cookie', b'__cf_bm=WlEYLL6sCJkJMfVVnfaYwXZFMrXMxNsGjlFcxk2vHas-1730914072-1.0.1.1-1t9Us4iiFsPUgudYF7zL6rRv5nT3v7aibd4jwh95yl

>>>
{}

Example:
Review: <The customer service was terrible. The product arrived late and was damaged.>
Result:
{
  "aspect": {
    "customer service": {
      "sentiment": "negative",
      "justification": "The customer service was terrible."
    },
    "product arrival": {
      "sentiment": "negative",
      "justification": "The product arrived late and was damaged."
    }
  }
}



Analyze the provided complaint delimited by <> and extract service-related
aspects, sentiments (positive/negative/neutral), and justifications.
Format the result as JSON object with the following structure: a key named 'aspect'
containing a nested object with keys for each aspect, and each aspect having an object with
'sentiment' and 'justification' keys.
Exclude non-product discussions. If no sentiment is expressed for an aspect, return
'neutral' as the sentiment and 'No information provided' as the value. If information
can't be retrieved, return an empty JSON object.
Review: <Managing an account Depo

DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Wed, 06 Nov 2024 17:27:54 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'user-nw5aseigyjbyzll3ouyazpnn'), (b'openai-processing-ms', b'1593'), (b'openai-version', b'2020-10-01'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'200000'), (b'x-ratelimit-remaining-requests', b'9998'), (b'x-ratelimit-remaining-tokens', b'199832'), (b'x-ratelimit-reset-requests', b'15.406s'), (b'x-ratelimit-reset-tokens', b'50ms'), (b'x-request-id', b'req_1225a1fd893b0b85b399e1696acd96b5'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'CF-Cache-Status', b'DYNAMIC'), (b'X-Content-Type-Options', b'nosniff'), (b'Server', b'cloudflare'), (b'CF-RAY', b'8de6cbfb1aec4366-EWR'), (b'Content-Encoding', 

...

>>>
{}

Example:
Review: <The customer service was terrible. The product arrived late and damaged.>
Result:
{
  "aspect": {
    "customer service": {
      "sentiment": "negative",
      "justification": "The customer service was terrible."
    },
    "product arrival": {
      "sentiment": "negative",
      "justification": "The product arrived late and damaged."
    }
  }
}



Analyze the provided complaint delimited by <> and extract service-related
aspects, sentiments (positive/negative/neutral), and justifications.
Format the result as JSON object with the following structure: a key named 'aspect'
containing a nested object with keys for each aspect, and each aspect having an object with
'sentiment' and 'justification' keys.
Exclude non-product discussions. If no sentiment is expressed for an aspect, return
'neutral' as the sentiment and 'No information provided' as the value. If information
can't be retrieved, return an empty JSON object.
Review: <Dealing with your lender or serv

DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Wed, 06 Nov 2024 17:28:24 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'user-nw5aseigyjbyzll3ouyazpnn'), (b'openai-processing-ms', b'1778'), (b'openai-version', b'2020-10-01'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'200000'), (b'x-ratelimit-remaining-requests', b'9984'), (b'x-ratelimit-remaining-tokens', b'199832'), (b'x-ratelimit-reset-requests', b'2m12.336s'), (b'x-ratelimit-reset-tokens', b'50ms'), (b'x-request-id', b'req_b2fed974d1a2485e60800d0f952a7a1a'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'CF-Cache-Status', b'DYNAMIC'), (b'X-Content-Type-Options', b'nosniff'), (b'Server', b'cloudflare'), (b'CF-RAY', b'8de6ccb6495e4366-EWR'), (b'Content-Encoding'

>>>
{}

Example:
Review: <The customer service was terrible. The product arrived late and was damaged.>
Result:
{
  "aspect": {
    "customer service": {
      "sentiment": "negative",
      "justification": "The customer service was terrible."
    },
    "product arrival": {
      "sentiment": "negative",
      "justification": "The product arrived late and was damaged."
    }
  }
}



Analyze the provided complaint delimited by <> and extract service-related
aspects, sentiments (positive/negative/neutral), and justifications.
Format the result as JSON object with the following structure: a key named 'aspect'
containing a nested object with keys for each aspect, and each aspect having an object with
'sentiment' and 'justification' keys.
Exclude non-product discussions. If no sentiment is expressed for an aspect, return
'neutral' as the sentiment and 'No information provided' as the value. If information
can't be retrieved, return an empty JSON object.
Review: <Managing an account Prob

DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Wed, 06 Nov 2024 17:28:26 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'user-nw5aseigyjbyzll3ouyazpnn'), (b'openai-processing-ms', b'1764'), (b'openai-version', b'2020-10-01'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'200000'), (b'x-ratelimit-remaining-requests', b'9983'), (b'x-ratelimit-remaining-tokens', b'199832'), (b'x-ratelimit-reset-requests', b'2m19.106s'), (b'x-ratelimit-reset-tokens', b'50ms'), (b'x-request-id', b'req_ae6f797fda853d0336dd17dd26807ba8'), (b'strict-transport-security', b'max-age=31536000; includeSubDomains; preload'), (b'CF-Cache-Status', b'DYNAMIC'), (b'X-Content-Type-Options', b'nosniff'), (b'Server', b'cloudflare'), (b'CF-RAY', b'8de6ccc20a9d4366-EWR'), (b'Content-Encoding'

>>>
{}

Example:
Review: <The customer service was terrible. The product arrived late and was damaged.>
Result:
{
  "aspect": {
    "customer service": {
      "sentiment": "negative",
      "justification": "The customer service was terrible."
    },
    "product arrival": {
      "sentiment": "negative",
      "justification": "The product arrived late and was damaged."
    }
  }
}


